# 🗣️ Chat with SQL Agent - 5 Minute Demo

This notebook demonstrates how to use the Chat with SQL Agent to interact with databases using natural language.

## 🚀 Quick Setup (< 2 minutes)

1. **Install dependencies**
2. **Set up OpenAI API key**
3. **Create sample database**
4. **Chat with your data!**

## Step 1: Install Dependencies

In [ ]:
# Install required packages
!pip install -q langchain openai pandas sqlite3 matplotlib seaborn plotly streamlit python-dotenv pydantic

## Step 2: Set Up Configuration

In [ ]:
import os
from getpass import getpass

# Set your OpenAI API key
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API key: ')

print("✅ Configuration set up successfully!")

## Step 3: Create Sample Database

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Create sample e-commerce database
conn = sqlite3.connect('demo_ecommerce.db')
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    country TEXT,
    signup_date DATE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    category TEXT,
    price REAL,
    stock_quantity INTEGER
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    order_date DATE,
    total_amount REAL,
    FOREIGN KEY (customer_id) REFERENCES customers (customer_id),
    FOREIGN KEY (product_id) REFERENCES products (product_id)
)
''')

# Generate sample data
np.random.seed(42)

# Sample customers
customers_data = [
    (1, 'Alice Johnson', 'alice@email.com', 'USA', '2023-01-15'),
    (2, 'Bob Smith', 'bob@email.com', 'Canada', '2023-02-20'),
    (3, 'Charlie Brown', 'charlie@email.com', 'UK', '2023-03-10'),
    (4, 'Diana Prince', 'diana@email.com', 'Australia', '2023-04-05'),
    (5, 'Eve Davis', 'eve@email.com', 'Germany', '2023-05-12')
]

cursor.executemany('INSERT OR REPLACE INTO customers VALUES (?, ?, ?, ?, ?)', customers_data)

# Sample products
products_data = [
    (1, 'Laptop', 'Electronics', 999.99, 50),
    (2, 'Smartphone', 'Electronics', 699.99, 100),
    (3, 'Coffee Mug', 'Home & Kitchen', 12.99, 200),
    (4, 'Running Shoes', 'Sports', 89.99, 75),
    (5, 'Book: Python Programming', 'Books', 29.99, 150),
    (6, 'Wireless Headphones', 'Electronics', 149.99, 80)
]

cursor.executemany('INSERT OR REPLACE INTO products VALUES (?, ?, ?, ?, ?)', products_data)

# Sample orders
orders_data = []
order_id = 1
start_date = datetime(2023, 6, 1)

for _ in range(50):
    customer_id = np.random.randint(1, 6)
    product_id = np.random.randint(1, 7)
    quantity = np.random.randint(1, 4)
    order_date = start_date + timedelta(days=np.random.randint(0, 180))
    
    # Get product price
    cursor.execute('SELECT price FROM products WHERE product_id = ?', (product_id,))
    price = cursor.fetchone()[0]
    total_amount = price * quantity
    
    orders_data.append((order_id, customer_id, product_id, quantity, order_date.strftime('%Y-%m-%d'), total_amount))
    order_id += 1

cursor.executemany('INSERT OR REPLACE INTO orders VALUES (?, ?, ?, ?, ?, ?)', orders_data)

conn.commit()
conn.close()

print("✅ Sample database created successfully!")
print("📊 Database contains:")
print("   • 5 customers")
print("   • 6 products")
print("   • 50 orders")

## Step 4: Set Up the SQL Agent

In [ ]:
import sys
import os

# Add the src directory to the path
sys.path.append('../src')

# Import our modules
from agents import create_sql_agent
from config import settings

# Create the SQL agent
database_url = "sqlite:///demo_ecommerce.db"
agent = create_sql_agent(database_url)

print("🤖 SQL Agent created successfully!")
print("Ready to chat with your database!")

## Step 5: Chat with Your Database! 🗣️

Now you can ask natural language questions about your data:

In [ ]:
# Example 1: Basic query
response = agent.run("How many customers do we have?")
print("Question: How many customers do we have?")
print(f"Answer: {response}")
print("\n" + "="*50 + "\n")

In [ ]:
# Example 2: Aggregation query
response = agent.run("What's the total revenue from all orders?")
print("Question: What's the total revenue from all orders?")
print(f"Answer: {response}")
print("\n" + "="*50 + "\n")

In [ ]:
# Example 3: Complex analysis
response = agent.run("Which product category generates the most revenue?")
print("Question: Which product category generates the most revenue?")
print(f"Answer: {response}")
print("\n" + "="*50 + "\n")

In [ ]:
# Example 4: Customer analysis
response = agent.run("Show me the top 3 customers by total spending")
print("Question: Show me the top 3 customers by total spending")
print(f"Answer: {response}")
print("\n" + "="*50 + "\n")

In [ ]:
# Example 5: Time-based analysis
response = agent.run("What was our average order value in the last 30 days?")
print("Question: What was our average order value in the last 30 days?")
print(f"Answer: {response}")
print("\n" + "="*50 + "\n")

## Step 6: Try Your Own Questions! 🎯

Use the cell below to ask your own questions:

In [ ]:
# Ask your own question!
your_question = "What are the most popular products by quantity sold?"

response = agent.run(your_question)
print(f"Question: {your_question}")
print(f"Answer: {response}")

## Step 7: Generate Reports 📊

In [ ]:
# Import reporting functions
from reporting import dataframe_to_plot, create_interactive_plot
import pandas as pd
import sqlite3

# Get some data for visualization
conn = sqlite3.connect('demo_ecommerce.db')

# Revenue by category
query = """
SELECT p.category, SUM(o.total_amount) as revenue
FROM orders o
JOIN products p ON o.product_id = p.product_id
GROUP BY p.category
ORDER BY revenue DESC
"""

df = pd.read_sql_query(query, conn)
conn.close()

print("📊 Revenue by Category:")
print(df)

# Create visualization
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.bar(df['category'], df['revenue'])
plt.title('Revenue by Product Category')
plt.xlabel('Category')
plt.ylabel('Revenue ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n✅ Visualization created!")

## 🎉 Congratulations!

You've successfully:

1. ✅ Set up the Chat with SQL Agent
2. ✅ Created a sample database
3. ✅ Asked natural language questions
4. ✅ Generated visualizations

## Next Steps:

- **Use with your own database**: Replace the sample database with your real data
- **Run the Streamlit app**: Use `streamlit run app/main.py` for a web interface
- **Customize the agent**: Modify `src/agents.py` to add custom tools
- **Add email reports**: Configure email settings in `.env` to send reports

## Try These Advanced Questions:

- "Create a monthly sales trend analysis"
- "Which customers haven't made a purchase in the last 60 days?"
- "What's the correlation between product price and sales volume?"
- "Generate a customer segmentation analysis"

Happy querying! 🚀